In [1]:
# Import all libraries
import pandas as pd
import numpy as np
import ast
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Load the datasets
movie=pd.read_csv(r"tmdb_5000_movies.csv")
credits=pd.read_csv(r"tmdb_5000_credits.csv")

In [3]:
#show first 10 records from movie dataset
movie.head(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,890871626,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576
6,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,591794936,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330
7,280000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/193/avenge

In [4]:
#show last 10 records from movie dataset
movie.tail(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4793,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,182291,"[{""id"": 718, ""name"": ""confession""}, {""id"": 100...",en,On The Downlow,Isaac and Angel are two young Latinos involved...,0.029757,"[{""name"": ""Iconoclast Films"", ""id"": 26677}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2004-04-11,0,90.0,[],Released,Two gangs. One secret. One crossroad.,On The Downlow,6.0,2
4794,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 27, ""n...",NaN,286939,[],en,Sanctuary: Quite a Conundrum,"It should have been just a normal day of sex, ...",0.166513,"[{""name"": ""Gold Lion Films"", ""id"": 37870}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-01-20,0,82.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Sanctuary: Quite a Conundrum,0.0,0
4795,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,124606,"[{""id"": 10726, ""name"": ""gang""}, {""id"": 33928, ...",en,Bang,A young woman in L.A. is having a bad day: she...,0.918116,"[{""name"": ""Asylum Films"", ""id"": 10571}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-09-09,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Sometimes you've got to break the rules,Bang,6.0,1
4796,7000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",http://www.primermovie.com,14337,"[{""id"": 1448, ""name"": ""distrust""}, {""id"": 2101...",en,Primer,Friends/fledgling entrepreneurs invent a devic...,23.307949,"[{""name"": ""Thinkfilm"", ""id"": 446}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2004-10-08,424760,77.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,What happens if it actually works?,Primer,6.9,658
4797,0,"[{""id"": 10769, ""name"": ""Foreign""}, {""id"": 53, ...",NaN,67238,[],en,Cavite,"Adam, a security guard, travels from Californi...",0.022173,[],[],2005-03-12,0,80.0,[],Released,NaN,Cavite,7.5,2
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-13,0,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6
4801,0,[],http://shanghaicalling.com/,126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-05-03,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7
4802,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,25975,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""Uni

In [5]:
#show first 10 records from credits dataset
credits.head(10)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,559,Spider-Man 3,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
6,38757,Tangled,"[{""cast_id"": 34, ""character"": ""Flynn Rider (vo...","[{""credit_id"": ""52fe46db9251416c91062101"", ""de..."
7,99861,Avengers: Age of Ultron,"[{""cast_id"": 76, ""character"": ""Tony Stark / Ir...","[{""credit_id"": ""55d5f7d4c3a3683e7e0016eb"", ""de..."
8,767,Harry Potter and the Half-Blood Prince,"[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...","[{""credit_id"": ""52fe4273c3a36847f801fab1"", ""de..."
9,209112,Batman v Superman: Dawn of Justice,"[{""cast_id"": 18, ""character"": ""Bruce Wayne / B...","[{""credit_id"": ""553bf23692514135c8002886"", ""de..."


In [6]:
#show last 10 records from credits dataset
credits.tail(10)

,movie_id,title,cast,crew
4793,182291,On The Downlow,"[{""cast_id"": 1, ""character"": ""Isaac"", ""credit_...","[{""credit_id"": ""548c416392514122ef00197d"", ""de..."
4794,286939,Sanctuary: Quite a Conundrum,"[{""cast_id"": 3, ""character"": ""Mimi"", ""credit_i...","[{""credit_id"": ""545f8107c3a3686cbb0041fd"", ""de..."
4795,124606,Bang,"[{""cast_id"": 2, ""character"": ""The Girl"", ""cred...","[{""credit_id"": ""52fe4ab0c3a368484e161add"", ""de..."
4796,14337,Primer,"[{""cast_id"": 1, ""character"": ""Aaron"", ""credit_...","[{""credit_id"": ""52fe45e79251416c75066791"", ""de..."
4797,67238,Cavite,[],"[{""credit_id"": ""52fe475dc3a368484e0c319f"", ""de..."
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."
4802,25975,My Date with Drew,"[{""cast_id"": 3, ""character"": ""Herself"", ""credi...","[{""credit_id"": ""58ce021b9251415a390165d9"", ""de..."


In [7]:
#shape of movie dataset
movie.shape

(4803, 20)

In [8]:
#shape of credits dataset
credits.shape

(4803, 4)

In [9]:
# information about movie dataset
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [10]:
# information about credits dataset
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


## DataPreprocessing

In [17]:
#Merging two dataset on 'title' based
movies=movie.merge(credits, on='title')
movies.title.head(50)

0                                          Avatar
1        Pirates of the Caribbean: At World's End
2                                         Spectre
3                           The Dark Knight Rises
4                                     John Carter
5                                    Spider-Man 3
6                                         Tangled
7                         Avengers: Age of Ultron
8          Harry Potter and the Half-Blood Prince
9              Batman v Superman: Dawn of Justice
10                               Superman Returns
11                              Quantum of Solace
12     Pirates of the Caribbean: Dead Man's Chest
13                                The Lone Ranger
14                                   Man of Steel
15       The Chronicles of Narnia: Prince Caspian
16                                   The Avengers
17    Pirates of the Caribbean: On Stranger Tides
18                                 Men in Black 3
19      The Hobbit: The Battle of the Five Armies


In [ ]:
# Select attributes use for recommendation
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
# show first 10 records of movies dataset
movies.head(10)

In [ ]:
# show last 10 records of movies dataset
movies.tail(10)

In [ ]:
#shape of movies dataset
movies.shape

In [ ]:
# information about movies dataset
movies.info()

In [ ]:
# Check null values
movies.isnull().sum()

In [ ]:
#remove null values
movies.dropna(inplace=True)

In [ ]:
# remove duplicated values
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

## Creating Model 

In [ ]:
def convert(obj):
    """
    Convert a string representation of a list of dictionaries into a list of names.
    
    Parameters:
        obj (str): A string containing a list of dictionaries.
        
    Returns:
        list: A list of names extracted from the dictionaries in the input string.
    """
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [ ]:
#apply the functions on "genres" and "keywords" attributes  
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies['keywords']

In [ ]:
movies['cast']

In [ ]:
def convert2(obj):
    """
    Converts a string representation of a list of dictionaries into a list of names.

    This function takes a string `obj` containing a representation of a list of dictionaries,
    where each dictionary contains a 'name' key. It extracts the 'name' values from the
    dictionaries and returns them in a list. The extraction process stops after the third
    'name' value is extracted.

    Parameters:
    obj (str): A string representation of a list of dictionaries.

    Returns:
    list: A list containing names extracted from dictionaries. The list will contain at
          most three names.
    """
    counter=0
    l=[]
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l

In [ ]:
#apply the functions on "cast" attributes 
movies['cast']=movies['cast'].apply(convert2)

In [ ]:
def find_director(obj):
    """
    Extracts the name of the director from a string representation of a list of crew members.

    This function takes a string `obj` containing a representation of a list of dictionaries,
    where each dictionary represents a crew member's information. It searches for a dictionary
    with the 'job' key set to 'Director' and returns the 'name' value associated with it.

    Parameters:
    obj (str): A string representation of a list of crew members' information.

    Returns:
    list: A list containing the name of the director. If no director is found, the list will
          be empty.
    """
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

In [ ]:
#apply the functions on "cast" attributes 
movies['crew']=movies['crew'].apply(find_director)

In [ ]:
# Split the 'overview' text into a list of words for each movie.
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
# Remove spaces from genre names in the 'genres' column
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
# Remove spaces from keyword names in the 'keywords' column
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
# Remove spaces from cast member names in the 'cast' column
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
# Remove spaces from crew member names in the 'crew' column
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
#After applying all the functions, display the dataset
movies

In [ ]:
# Combine cast, overview, genres, keywords, and crew information to create 'tags' column
movies['tags']=movies['cast']+movies['overview']+movies['genres']+movies['keywords']+movies['crew']

In [ ]:
#Create a new DataFrame 'new_movies' containing only 'movie_id', 'title', and 'tags' columns
new_movies=movies[['movie_id', 'title', 'tags']]

In [ ]:
new_movies

In [ ]:
# Concatenate the list elements in the 'tags' column into a single string for each row in 'new_movies'
new_movies['tags']=new_movies['tags'].apply(lambda x: " ".join(x))

In [ ]:
new_movies['tags'][0]

In [ ]:
new_movies['tags'][1]

### convert text to vector using "Bags of words" for similarity content of movies

In [ ]:
# Create a CountVectorizer object with a maximum of 5000 features and excluding English stop words
cv = CountVectorizer(max_features=5000,stop_words='english')
#transform the 'tags' column in 'new_movies' into a sparse matrix of word counts
vect =cv.fit_transform(new_movies['tags']).toarray()

In [ ]:
vect

In [ ]:
# Create a PorterStemmer object for word stemming
ps= PorterStemmer()

In [ ]:
def stem_function(text):
    """
    Applies stemming using Porter Stemmer to input text.

    This function takes a string of text, splits it into individual words, applies Porter Stemming
    to each word, and then joins the stemmed words back into a single string.

    Parameters:
    text (str): The input text to be stemmed.

    Returns:
    str: The input text with words stemmed using Porter Stemming.
    """
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)   

In [ ]:
#apply "Stem function" on "tags" feature
new_movies['tags']=new_movies['tags'].apply(stem_function)

In [ ]:
#Calculate the cosine similarity matrix
cosine_similarity(vect)

In [ ]:
# Get the shape of the cosine similarity matrix
cosine_similarity(vect).shape

In [ ]:
# Calculate and store the cosine similarity matrix in the 'similarity' variable
similarity=cosine_similarity(vect)
similarity

In [ ]:
#1st movie similarity
print(similarity[0])
#2nd movie similarity
print(similarity[1])
#last movie similarity
print(similarity[4805])

In [ ]:
# Sort and retrieve the indices of top 5 most similar movies to the first movie in the similarity matrix
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

In [ ]:
# find information of movie using movie index
new_movies.iloc[539]

In [ ]:
def recommend(movie):
    """
    Recommends similar movies based on input movie title.

    This function takes a movie title, finds its index in the 'new_movies' DataFrame,
    calculates cosine similarity distances between the input movie and all other movies,
    sorts the distances, and then prints the titles of the top 5 most similar movies.

    Parameters:
    movie (str): The title of the movie for which recommendations are sought.

    Returns:
    None
    """
    movie_index=new_movies[new_movies['title'] == movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_movies.iloc[i[0]].title)
        

In [ ]:
# Recommend similar movies to the movie 'Avatar'
recommend('Avatar')

In [ ]:
# Serialize and save the 'new_movies' DataFrame as a dictionary in a pickle file
pickle.dump(new_movies.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
# Serialize and save the cosine similarity matrix as a pickle file
pickle.dump(similarity,open('similarity.pkl','wb'))